In [29]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

In [30]:
df = pd.read_csv("penguins.csv")

In [31]:
total_penguins = len(df)
avg_bill_length = df['bill_length_mm'].mean()
avg_body_mass = df['body_mass_g'].mean()
most_common_species = df['species'].mode()[0]

In [32]:
app = dash.Dash(__name__)

In [33]:
app.layout = html.Div(children=[
    
    # Title
    html.H1("Penguin Data Visualization Dashboard", style={'textAlign': 'center', 'color': '#4A90E2'}),
    
    html.Div([
        html.Div([
            html.H3("Total Penguins", style={'textAlign': 'center'}),
            html.P(f"{total_penguins}", style={'textAlign': 'center', 'fontSize': '24px'})
        ], className='metric-box', style={'padding': '10px', 'backgroundColor': '#F0F8FF', 'borderRadius': '10px', 'margin': '10px'}),
        
        html.Div([
            html.H3("Average Bill Length (mm)", style={'textAlign': 'center'}),
            html.P(f"{avg_bill_length:.2f}", style={'textAlign': 'center', 'fontSize': '24px'})
        ], className='metric-box', style={'padding': '10px', 'backgroundColor': '#F0F8FF', 'borderRadius': '10px', 'margin': '10px'}),
        
        html.Div([
            html.H3("Average Body Mass (g)", style={'textAlign': 'center'}),
            html.P(f"{avg_body_mass:.2f}", style={'textAlign': 'center', 'fontSize': '24px'})
        ], className='metric-box', style={'padding': '10px', 'backgroundColor': '#F0F8FF', 'borderRadius': '10px', 'margin': '10px'}),
        
        html.Div([
            html.H3("Most Common Species", style={'textAlign': 'center'}),
            html.P(f"{most_common_species}", style={'textAlign': 'center', 'fontSize': '24px'})
        ], className='metric-box', style={'padding': '10px', 'backgroundColor': '#F0F8FF', 'borderRadius': '10px', 'margin': '10px'})
    ], className='metrics-container', style={'display': 'flex', 'justifyContent': 'center'}),
    
    # Filters Section
    html.Div([
        html.Label('Select Species', style={'fontWeight': 'bold'}),
        dcc.Dropdown(
            id='species-filter',
            options=[{'label': species, 'value': species} for species in df['species'].unique()],
            value=df['species'].unique().tolist(),  # Default to all species
            multi=True,
            style={'marginBottom': '10px'}
        ),
        
        html.Label('Select Island', style={'fontWeight': 'bold'}),
        dcc.Dropdown(
            id='island-filter',
            options=[{'label': island, 'value': island} for island in df['island'].unique()],
            value=df['island'].unique().tolist(),  # Default to all islands
            multi=True,
            style={'marginBottom': '10px'}
        ),
        
        html.Label('Select Year', style={'fontWeight': 'bold'}),
        dcc.Dropdown(
            id='year-filter',
            options=[{'label': int(year), 'value': year} for year in df['year'].unique()],
            value=df['year'].unique().tolist(),  # Default to all years
            multi=True,
            style={'marginBottom': '10px'}
        ),
    ], className='filter-container', style={'padding': '20px'}),
    
    # Graph 1: Bill Length vs Bill Depth
    html.Div([
        dcc.Graph(id='scatter-plot'),
    ], style={'padding': '20px'}),
    
    # Graph 2: Flipper Length vs Body Mass
    html.Div([
        dcc.Graph(id='scatter-plot2'),
    ], style={'padding': '20px'}),
    
    # Graph 3: Bar plot of Average Body Mass by Species
    html.Div([
        dcc.Graph(id='bar-plot'),
    ], style={'padding': '20px'}),
    
    # Graph 4: Pie chart of distribution by island
    html.Div([
        dcc.Graph(id='pie-chart'),
    ], style={'padding': '20px'}),

    # Footer
    html.Div([
        html.Hr(),
        html.P("Developed by Surabhi Jaiswal", style={'textAlign': 'center', 'fontSize': '18px', 'color': '#888'})
    ], style={'marginTop': '30px'})
])

In [34]:
@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('scatter-plot2', 'figure'),
     Output('bar-plot', 'figure'),
     Output('pie-chart', 'figure')],
    [Input('species-filter', 'value'),
     Input('island-filter', 'value'),
     Input('year-filter', 'value')]
)
def update_graphs(selected_species, selected_island, selected_year):
    filtered_df = df[(df['species'].isin(selected_species)) & 
                     (df['island'].isin(selected_island)) & 
                     (df['year'].isin(selected_year))]
    
    # Scatter plot: Bill Length vs Bill Depth
    fig_scatter = px.scatter(
        filtered_df, x='bill_length_mm', y='bill_depth_mm', 
        color='species', 
        title="Bill Length vs Bill Depth"
    )
    
    # Scatter plot: Flipper Length vs Body Mass
    fig_scatter2 = px.scatter(
        filtered_df, x='flipper_length_mm', y='body_mass_g', 
        color='species', 
        title="Flipper Length vs Body Mass"
    )
    
    # Bar plot: Average Body Mass by Species
    avg_body_mass_by_species = filtered_df.groupby('species')['body_mass_g'].mean().reset_index()
    fig_bar = px.bar(
        avg_body_mass_by_species, 
        x='species', y='body_mass_g', 
        title="Average Body Mass by Species"
    )
    
    # Pie chart: Distribution by Island
    fig_pie = px.pie(
        filtered_df, names='island', 
        title="Penguin Distribution by Island"
    )
    
    return fig_scatter, fig_scatter2, fig_bar, fig_pie

In [35]:
import warnings

In [36]:
warnings.filterwarnings("ignore")

In [37]:
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)